<a href="https://colab.research.google.com/github/jerryk42/SemEval-Food-Hazard-Detection-Challenge-2025/blob/main/Submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup and Configuration


In [ ]:
# Importing all the necessary libraries
from google.colab import drive
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import LabelEncoder
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import re

# Data Loading and Preprocessing

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Configuration of the model
config = {
    'max_len': 256,
    'batch_size': 16,
    'learning_rate': 0.00005,
    'epochs': 50,
    'early_stopping_patience': 6,  # Early stopping patience
    'model_name': "dmis-lab/biobert-base-cased-v1.1"  # BioBERT (better results than PubBERT and BioGPT)
}


# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Data Loading and Preprocessing


In [ ]:
# Load labeled training data
url = "https://raw.githubusercontent.com/food-hazard-detection-semeval-2025/food-hazard-detection-semeval-2025.github.io/refs/heads/main/data/incidents_train.csv"
df_train = pd.read_csv(url)
df_train = df_train.drop(df_train.columns[0], axis=1)


# Preprocess on the text (cleaning)
def clean_text(text):
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = text.lower()
    return ' '.join(text.split())

# Clean the text data in the training DataFrame
df_train['text'] = df_train['text'].apply(clean_text)

# Load unlabeled validation data
unlabeled_path = '/content/drive/My Drive/AUEB/incidents.csv'
df_unlabeled = pd.read_csv(unlabeled_path)

# Clean the text data in the validation DataFrame
df_unlabeled['text'] = df_unlabeled['text'].apply(clean_text)

# Tokenization

In [ ]:
# Using AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(config['model_name'])

# Dataset Class

In [ ]:
# Custom Dataset Class
class TextDataset(Dataset):
    def __init__(self, texts, tokenizer, max_len, labels=None):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        item = {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten()
        }
        if self.labels is not None:
            item['label'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

# Label Encoding

In [ ]:
# Encode labels and split the tasks
targets_st1 = ['hazard-category', 'product-category']
targets_st2 = ['hazard', 'product']

# Encode labels for all targets
label_encoders = {}
for target in targets_st1 + targets_st2:
    le = LabelEncoder()
    df_train[target] = le.fit_transform(df_train[target])
    label_encoders[target] = le

# Prepare training and validation data
train_texts = df_train['text'].values # training data
val_texts = df_unlabeled['text'].values # validation data

# Training and Validation

In [ ]:
# Train and validate for each target
for target in targets_st1 + targets_st2:
    print(f"\nTraining for target: {target}")
    train_labels = df_train[target].values
    train_dataset = TextDataset(train_texts, tokenizer, config['max_len'], train_labels)
    val_dataset = TextDataset(val_texts, tokenizer, config['max_len'])

    train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False)

    # Initialize the model for sequence classification
    num_labels = len(label_encoders[target].classes_)
    model = AutoModelForSequenceClassification.from_pretrained(config['model_name'], num_labels=num_labels).to(device)

    optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])
    criterion = nn.CrossEntropyLoss()
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

    best_loss = float('inf')
    early_stop_counter = 0

    # Training loop
    for epoch in range(config['epochs']):
        print(f"Epoch {epoch+1}/{config['epochs']} - Target: {target}")
        model.train()
        total_loss = 0

        for batch in tqdm(train_loader):
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            loss = criterion(outputs.logits, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        print(f"Average Training Loss: {avg_loss}")

        # Validation loop
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                outputs = model(input_ids, attention_mask=attention_mask)

        scheduler.step(avg_loss)

        # Early stopping mechanism
        if avg_loss < best_loss:
            best_loss = avg_loss
            early_stop_counter = 0
            torch.save(model.state_dict(), f'/content/drive/My Drive/AUEB/best_model_{target}.pt')
        else:
            early_stop_counter += 1
            print(f"Early stopping counter: {early_stop_counter}/{config['early_stopping_patience']}")

        if early_stop_counter >= config['early_stopping_patience']:
            print(f"Early stopping triggered for {target}.")
            break

    # Generate predictions on the validation dataset
    print(f"\nGenerating predictions for target: {target}")
    model.load_state_dict(torch.load(f'/content/drive/My Drive/AUEB/best_model_{target}.pt'))
    model.eval()
    predictions = []
    with torch.no_grad():
        for batch in tqdm(val_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            predictions.extend(preds.cpu().numpy())

    # Save predictions for each target
    df_unlabeled[f'predicted_{target}'] = label_encoders[target].inverse_transform(predictions)

# Save ST1 and ST2 predictions separately
st1_output_path = '/content/drive/My Drive/AUEB/ST1_predictions.csv'
st2_output_path = '/content/drive/My Drive/AUEB/ST2_predictions.csv'

df_st1 = df_unlabeled[['text', 'predicted_hazard-category', 'predicted_product-category']]
df_st2 = df_unlabeled[['text', 'predicted_hazard', 'predicted_product']]

df_st1.to_csv(st1_output_path, index=False)
df_st2.to_csv(st2_output_path, index=False)

print(f"ST1 predictions saved to {st1_output_path}")
print(f"ST2 predictions saved to {st2_output_path}")

Mounted at /content/drive
Using device: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]


Training for target: hazard-category


...

Average Training Loss: 0.01098893606982563
Epoch 60/60 - Target: product


100%|██████████| 318/318 [01:44<00:00,  3.05it/s]


Average Training Loss: 0.010448389430750021

Generating predictions for target: product


<ipython-input-3-4184609b97d1>:148: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'/content/drive/My Drive/AUEB/best_model_{target}.pt'))
1

ST1 predictions saved to /content/drive/My Drive/AUEB/ST1_predictions.csv
ST2 predictions saved to /content/drive/My Drive/AUEB/ST2_predictions.csv


# Output Correction


Correction of the two seperate .csv files for subtask 1 and subtask 2 according to the competition's directions.

In [ ]:
# File paths
st1_path = '/content/drive/My Drive/AUEB/ST1_predictions.csv'
st2_path = '/content/drive/My Drive/AUEB/ST2_predictions.csv'

# Save paths
st1_output_path = '/content/drive/My Drive/AUEB/ST1_predictions_cleaned.csv'
st2_output_path = '/content/drive/My Drive/AUEB/ST2_predictions_cleaned.csv'

# Process ST1_predictions.csv
df_st1 = pd.read_csv(st1_path)  # Load the file
df_st1 = df_st1.iloc[:, 1:]  # Drop the first column
df_st1.columns = ['hazard-category', 'product-category']  # Rename columns
df_st1.to_csv(st1_output_path, index=False)  # Save back to Google Drive
print(f"Processed ST1_predictions.csv saved to: {st1_output_path}")

# Process ST2_predictions.csv
df_st2 = pd.read_csv(st2_path)  # Load the file
df_st2 = df_st2.iloc[:, 1:]  # Drop the first column
df_st2.columns = ['hazard', 'product']  # Rename columns
df_st2.to_csv(st2_output_path, index=False)  # Save back to Google Drive
print(f"Processed ST2_predictions.csv saved to: {st2_output_path}")

Processed ST1_predictions.csv saved to: /content/drive/My Drive/AUEB/ST1_predictions_cleaned.csv
Processed ST2_predictions.csv saved to: /content/drive/My Drive/AUEB/ST2_predictions_cleaned.csv
